# HurdleDMR

HurdleDMR.jl is a Julia implementation of the Hurdle Distributed Multinomial Regression (HDMR), as described in:

Kelly, Bryan, Asaf Manela, and Alan Moreira (2018). Text Selection. [Working paper](http://apps.olin.wustl.edu/faculty/manela/kmm/textselection/).

It includes a Julia implementation of the Distributed Multinomial Regression (DMR) model of [Taddy (2015)](https://arxiv.org/abs/1311.6139).

This tutorial explains how to use this package.

## Setup

### Install Julia

First, install Julia itself. The easiest way to do that is from the download site https://julialang.org/downloads/. An alternative is to install JuliaPro from https://juliacomputing.com

Once installed, open julia in a terminal (or in Juno), press `]` to activate package manager and add the following packages:
```
pkg> add HurdleDMR GLM Lasso
```

### Install PyJulia
See the documentation [here](https://github.com/JuliaPy/pyjulia) for installation instructions.

Because I use miniconda, I also had to run the following, but you might not:

In [1]:
from julia.api import Julia
jl = Julia(compiled_modules=False)

In [4]:
%load_ext julia.magic

The julia.magic extension is already loaded. To reload it, use:
  %reload_ext julia.magic


Add parallel workers and make package available to workers

In [5]:
%julia using Distributed
%julia addprocs(4)
%julia import HurdleDMR; @everywhere using HurdleDMR

### Example Data

Setup your data into an n-by-p covars matrix, and a (sparse) n-by-d counts matrix. Here we generate some random data.

In [65]:
import numpy as np
from scipy import sparse

n = 100
p = 3
d = 4

np.random.seed(13)
m = 1 + np.random.poisson(5,n)
covars = np.random.uniform(0,1,(n,p))

q = [[0 + j*sum(covars[i,:]) for j in range(d)] for i in range(n)]
#rowsums = [sum(q[i]) for i in range(n)]
q = [q[i]/sum(q[i]) for i in range(n)]

#counts = sparse.csr_matrix(np.concatenate([[np.random.multinomial(m[i],q[i]) for i in range(n)]]))
counts = np.concatenate([[np.random.multinomial(m[i],q[i]) for i in range(n)]])
counts

array([[0, 1, 5, 5],
       [0, 1, 1, 2],
       [0, 0, 2, 2],
       [0, 1, 0, 5],
       [0, 0, 1, 3],
       [0, 0, 3, 5],
       [0, 3, 2, 2],
       [0, 0, 4, 2],
       [0, 2, 2, 0],
       [0, 0, 2, 3],
       [0, 0, 5, 1],
       [0, 0, 1, 6],
       [0, 1, 3, 2],
       [0, 0, 4, 3],
       [0, 1, 0, 4],
       [0, 0, 3, 1],
       [0, 2, 0, 2],
       [0, 3, 1, 4],
       [0, 0, 2, 2],
       [0, 0, 1, 3],
       [0, 1, 1, 1],
       [0, 2, 4, 2],
       [0, 2, 4, 0],
       [0, 0, 2, 1],
       [0, 0, 3, 4],
       [0, 2, 2, 1],
       [0, 1, 1, 2],
       [0, 2, 0, 2],
       [0, 1, 2, 2],
       [0, 0, 1, 4],
       [0, 3, 1, 5],
       [0, 0, 2, 3],
       [0, 1, 2, 3],
       [0, 0, 3, 2],
       [0, 0, 2, 3],
       [0, 1, 2, 3],
       [0, 0, 2, 3],
       [0, 2, 1, 5],
       [0, 0, 2, 4],
       [0, 3, 3, 2],
       [0, 2, 0, 3],
       [0, 2, 1, 3],
       [0, 3, 0, 4],
       [0, 1, 3, 1],
       [0, 0, 2, 2],
       [0, 1, 1, 3],
       [0, 1, 2, 1],
       [0, 0,

## Distributed Multinomial Regression (DMR)

The Distributed Multinomial Regression (DMR) model of Taddy (2015) is a highly scalable
approximation to the Multinomial using distributed (independent, parallel)
Poisson regressions, one for each of the d categories (columns) of a large `counts` matrix,
on the `covars`.

To fit a DMR:

In [75]:
from julia import HurdleDMR as hd
m = hd.dmr(covars, counts)

or with a dataframe and formula

in either case we can get the coefficients matrix for each variable + intercept as usual with

In [76]:
hd.coef(m)

array([[ 0.        , -1.81259356, -1.15145269, -0.68629784],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.10842935,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ]])

By default we only return the AICc maximizing coefficients.
To also get back the entire regulatrization paths, run

In [77]:
paths = hd.dmrpaths(covars, counts)

we can now select, for example the coefficients that minimize 10-fold CV mse (takes a while)

In [81]:
%julia using Lasso: MinCVmse
gen = %julia MinCVKfold{MinCVmse}(10)
hd.coef(paths, gen)

array([[ 0.00000000e+00, -1.81259356e+00, -1.09861236e+00,
        -6.86297839e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.46603305e-07,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00]])

## Hurdle Distributed Multinomial Regression (HDMR)

For highly sparse counts, as is often the case with text that is selected for
various reasons, the Hurdle Distributed Multinomial Regression (HDMR) model of
Kelly, Manela, and Moreira (2018), may be superior to the DMR. It approximates
a higher dispersion Multinomial using distributed (independent, parallel)
Hurdle regressions, one for each of the d categories (columns) of a large `counts` matrix,
on the `covars`. It allows a potentially different sets of covariates to explain
category inclusion ($h=1{c>0}$), and repetition ($c>0$).

Both the model for zeroes and for positive counts are regularized by default,
using `GammaLassoPath`, picking the AICc optimal segment of the regularization
path.

HDMR can be fitted:

In [8]:
m = hdmr(covars, counts; inpos=1:2, inzero=1:3)

┌ Info: fitting 100 observations on 4 categories 
│ 2 covariates for positive and 3 for zero counts
└ @ HurdleDMR /home/root/.julia/dev/HurdleDMR/src/hdmr.jl:468
┌ Info: distributed InclusionRepetition run on local cluster with 4 nodes
└ @ HurdleDMR /home/root/.julia/dev/HurdleDMR/src/hdmr.jl:481


HDMRCoefs{InclusionRepetition,Array{Float64,2},MinAICc,UnitRange{Int64}}([0.0 -2.48491 -0.505547 -0.23561; 0.0 0.0 -2.65019 0.203207; 0.0 0.0 -0.491465 0.0], [-2.59356 1.23117 1.73801 -13.9022; 0.0 -2.17746 0.0 38.1537; 0.0 -2.89271 0.0 423.955; 0.0 -1.23601 -1.85064 -19.9601], true, 100, 4, 1:2, 1:3, MinAICc(2))

or with a dataframe and formula

In [9]:
mf = @model(h ~ vy + v1 + v2, c ~ vy + v1)
m = fit(HDMR, mf, covarsdf, counts)

┌ Info: fitting 100 observations on 4 categories 
│ 2 covariates for positive and 3 for zero counts
└ @ HurdleDMR /home/root/.julia/dev/HurdleDMR/src/hdmr.jl:468
┌ Info: distributed InclusionRepetition run on local cluster with 4 nodes
└ @ HurdleDMR /home/root/.julia/dev/HurdleDMR/src/hdmr.jl:481


HDMRCoefs{InclusionRepetition,Array{Float64,2},MinAICc,Array{Int64,1}}([0.0 -2.48491 -0.505547 -0.23561; 0.0 0.0 -2.65019 0.203207; 0.0 0.0 -0.491465 0.0], [-2.59356 1.23117 1.73801 -13.9022; 0.0 -2.17746 0.0 38.1537; 0.0 -2.89271 0.0 423.955; 0.0 -1.23601 -1.85064 -19.9601], true, 100, 4, [1, 2], [1, 2, 3], MinAICc(2))

where the h ~ equation is the model for zeros (hurdle crossing) and c ~ is the model for positive counts

in either case we can get the coefficients matrix for each variable + intercept as usual with

In [10]:
coefspos, coefszero = coef(m)

([0.0 -2.48491 -0.505547 -0.23561; 0.0 0.0 -2.65019 0.203207; 0.0 0.0 -0.491465 0.0], [-2.59356 1.23117 1.73801 -13.9022; 0.0 -2.17746 0.0 38.1537; 0.0 -2.89271 0.0 423.955; 0.0 -1.23601 -1.85064 -19.9601])

By default we only return the AICc maximizing coefficients.
To also get back the entire regulatrization paths, run

In [11]:
paths = fit(HDMRPaths, mf, covarsdf, counts)

coef(paths, AllSeg())

┌ Info: fitting 100 observations on 4 categories 
│ 2 covariates for positive and 3 for zero counts
└ @ HurdleDMR /home/root/.julia/dev/HurdleDMR/src/hdmr.jl:281
┌ Info: distributed InclusionRepetition run on remote cluster with 4 nodes
└ @ HurdleDMR /home/root/.julia/dev/HurdleDMR/src/hdmr.jl:312


([0.0 0.0 0.0; 0.0 0.0 0.0; … ; 0.0 0.0 0.0; 0.0 0.0 0.0]

[-2.48491 0.0 0.0; -2.41917 -0.199534 0.0; … ; 0.0 0.0 0.0; 0.0 0.0 0.0]

[-1.78805 0.0 0.0; -1.69583 -0.197051 0.0; … ; 0.0 0.0 0.0; 0.0 0.0 0.0]

[-0.133883 3.92431e-7 0.0; -0.149695 0.0319783 0.0; … ; 0.0 0.0 0.0; 0.0 0.0 0.0], [-2.59356 0.0 0.0 0.0; -2.54089 0.0 0.0 -0.121014; … ; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]

[-1.5251 0.0 0.0 0.0; -1.43208 0.0 -0.192178 0.0; … ; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]

[0.670484 0.0 0.0 -4.29912e-8; 0.762441 0.0 0.0 -0.163359; … ; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]

[4.94744 0.0 0.0 0.0; 4.64738 0.0 0.592183 0.0; … ; -13.6801 37.63 417.646 -19.6945; -13.9022 38.1537 423.955 -19.9601])

## Sufficient reduction projection

A sufficient reduction projection summarizes the counts, much like a sufficient
statistic, and is useful for reducing the d dimensional counts in a potentially
much lower dimension matrix `z`.

To get a sufficient reduction projection in direction of vy for the above
example

In [12]:
z = srproj(m,counts,1,1)

100×4 Array{Float64,2}:
 -0.367472   19.0768   10.0  2.0
 -0.0821325  19.0768   10.0  2.0
  0.203207   38.1537    3.0  1.0
 -0.158994   11.9921    9.0  3.0
  0.135472   11.9921    6.0  3.0
  0.203207   38.1537    7.0  1.0
 -0.670108   11.9921    7.0  3.0
 -0.204421   19.0768    7.0  2.0
 -0.153467   19.0768    8.0  2.0
 -0.430881   19.0768    9.0  2.0
 -0.113837   19.0768    9.0  2.0
  0.203207   38.1537    4.0  1.0
  0.203207   38.1537    4.0  1.0
  ⋮                             
 -1.01943     8.99406   5.0  4.0
  0.203207   38.1537    4.0  1.0
 -1.22349    19.0768    4.0  2.0
 -0.747925   19.0768    6.0  2.0
 -0.367472   19.0768    5.0  2.0
 -0.510142   19.0768    4.0  2.0
 -0.315592   19.0768   11.0  2.0
 -0.233451   11.9921    7.0  3.0
 -1.24889    12.7179    8.0  3.0
 -1.22349    19.0768    2.0  2.0
 -1.22349    19.0768    6.0  2.0
 -0.612049   19.0768    7.0  2.0

Here, the first column is the SR projection from the model for positive counts, the second is the the SR projection from the model for hurdle crossing (zeros), and the third is the total count for each observation.

## Counts Inverse Regression (CIR)

Counts inverse regression allows us to predict a covariate with the counts and other covariates.
Here we use hdmr for the backward regression and another model for the forward regression.
This can be accomplished with a single command, by fitting a CIR{HDMR,FM} where the forward model is FM <: RegressionModel.

In [13]:
cir = fit(CIR{HDMR,LinearModel},mf,covarsdf,counts,:vy; nocounts=true)

┌ Info: fitting 100 observations on 4 categories 
│ 2 covariates for positive and 3 for zero counts
└ @ HurdleDMR /home/root/.julia/dev/HurdleDMR/src/hdmr.jl:468
┌ Info: distributed InclusionRepetition run on local cluster with 4 nodes
└ @ HurdleDMR /home/root/.julia/dev/HurdleDMR/src/hdmr.jl:481


CIR{HDMR,LinearModel}(1, [1, 2], HDMRCoefs{InclusionRepetition,Array{Float64,2},MinAICc,Array{Int64,1}}([0.0 -2.48491 -0.505547 -0.23561; 0.0 0.0 -2.65019 0.203207; 0.0 0.0 -0.491465 0.0], [-2.59356 1.23117 1.73801 -13.9022; 0.0 -2.17746 0.0 38.1537; 0.0 -2.89271 0.0 423.955; 0.0 -1.23601 -1.85064 -19.9601], true, 100, 4, [1, 2], [1, 2, 3], MinAICc(2)), LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,Cholesky{Float64,Array{Float64,2}}}}:

Coefficients:
─────────────────────────────────────────────────────────────────────────
       Estimate  Std. Error    t value  Pr(>|t|)    Lower 95%   Upper 95%
─────────────────────────────────────────────────────────────────────────
x1   0.806879    0.254456     3.171       0.0021   0.30158     1.31218   
x2  -0.103535    0.096488    -1.07304     0.2860  -0.295142    0.0880707 
x3  -0.115557    0.103506    -1.11642     0.2671  -0.321099    0.0899861 
x4   0.154634    0.0814074    1.89951     0.0606  -0.00702506  0.316293  
x5  -0

where the ```nocounts=true``` means we also fit a benchmark model without counts.

we can get the forward and backward model coefficients with

In [14]:
coefbwd(cir)

([0.0 -2.48491 -0.505547 -0.23561; 0.0 0.0 -2.65019 0.203207; 0.0 0.0 -0.491465 0.0], [-2.59356 1.23117 1.73801 -13.9022; 0.0 -2.17746 0.0 38.1537; 0.0 -2.89271 0.0 423.955; 0.0 -1.23601 -1.85064 -19.9601])

In [15]:
coeffwd(cir)

7-element Array{Float64,1}:
  0.8068791910919334   
 -0.10353546049021634  
 -0.11555664427731226  
  0.1546339871132811   
 -0.0010829538468236821
  0.005881340629811088 
 -0.0905476479689421   

The fitted model can be used to predict vy with new data

In [16]:
yhat = predict(cir, covarsdf[1:10,:], counts[1:10,:])

10-element Array{Union{Missing, Float64},1}:
 0.4698993353328307 
 0.45963979129838384
 0.5782657496603597 
 0.484620220504942  
 0.5023887465911144 
 0.5914871966842338 
 0.40669982045527   
 0.5011220105167319 
 0.4811118616060238 
 0.5030932551174164 

We can also predict only with the other covariates, which in this case
is just a linear regression

In [17]:
yhat_nocounts = predict(cir, covarsdf[1:10,:], counts[1:10,:]; nocounts=true)

10-element Array{Union{Missing, Float64},1}:
 0.45731660084749226
 0.5179985510761541 
 0.4807768729723196 
 0.47254808567420575
 0.47267552235044974
 0.48519627697522144
 0.4799151241219515 
 0.44186379731084846
 0.4592317510593359 
 0.4232136851484049 

Kelly, Manela, and Moreira (2018) show that the differences between DMR and HDMR can be substantial in some cases, especially when the counts data is highly sparse.

Please reference the paper for additional details and example applications.